In [63]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import string
import json
from sklearn.utils import shuffle

In [ ]:
# test cell
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
# nltk downloads
nltk.download('stopwords')
nltk.download('punkt')

In [66]:
# Changed to load from json files

all_path = "../data/500QA.json"
with open(all_path) as f:
  all_data = json.load(f)


Stemming and stopword removal functions


In [67]:
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words('english') + list(string.punctuation))

def clean_text(text):
  lst_tokens = [i for i in word_tokenize(text.lower()) if i not in stop]
  lemmatized_lst = []
  for token in lst_tokens:
    lemmatized_token = lemmatizer.lemmatize(token)
    lemmatized_lst.append(lemmatized_token)
  lemmatized_sentence = " ".join(lemmatized_lst)
  return lemmatized_sentence

In [68]:
all_descriptions = []
queries = []
for d in all_data:
  queries.append(d["query"])
  d["query"] = clean_text(d["query"])
  for o in d["options"]:
    d["options"][o] = clean_text(d["options"][o])
    all_descriptions += d["options"][o]

descriptions = all_descriptions

Choose correct option based on highest word overlap between query and description

In [71]:
# test cell

# Helper function to find number of overlapping words between two sentences
def calc_overlap(s1, s2):
  s1_list = s1.split(' ')
  s2_list = s2.split(' ')
  return len(list(set(s1_list)&set(s2_list)))

# General function to determine correct answer based on word overlap after pre-processing
def word_overlap(data, name):
  results = []
  correct = 0
  total = len(data)
  type_correct = {
    "Specific": 0,
    "Subjective": 0,
    "Indirect": 0,
    "Compound": 0,
    "Negated": 0,
    "Analogical": 0,
    "Temporal": 0}
    
  for d in data:
    try:
      options = [val for val in d['options'].values()]
      query = d['query']
      answer = d['options'][d['answer']]
    except:
      continue
    
    options_str = [str(i) for i in options]
    cleaned_query = str(query)
    
    overlap = []
    for option in options_str:
      cleaned_option = option
      num_overlap = calc_overlap(cleaned_query, cleaned_option)
      overlap.append(num_overlap)
    
    overlap, options = shuffle(overlap, options, random_state=0)
    # Ties in argmax are broken by picking first element
    ind = np.argmax(overlap) 
    result = 0
    if (options[ind]) == answer:
      correct += 1
      result = 1
      for key in d['query_type']:
        if d['query_type'][key] == 1:
          type_correct[key] += 1
    #results.append([result, "Query: " + query, "Recommended: " + str(options[ind])])
  print("Results for {}:".format(name))
  print("Total correct answers: {} out of {}".format(correct, total))
  #print(results)
  print(type_correct)

In [ ]:
word_overlap(all_data, "all")